In [ ]:
%pylab inline
import os

rc('font',**{'family':'serif',
             'serif':['Palatino']})
rc('text', **{'usetex': True})

In [ ]:
execfile("/Users/fdu/Dropbox/myScripts/my_script.py")
execfile("/Users/fdu/Dropbox/myScripts/long_function_definitions.py")
mycm = make_my_colormap(c_list=[(0.0, (0.5, 0.0, 0.5)),
                                (0.2, (0.0, 0.0, 1.0)),
                                (0.4, (0.0, 0.8, 1.0)),
                                (0.6, (0.0, 0.8, 0.0)),
                                (0.8, (1.0, 0.8, 0.0)),
                                (1.0, (1.0, 0.0, 0.0))])

rcParams['axes.color_cycle'] = mycolors
#print mycolors

In [ ]:
def lower_keys(p):
    for k in p.keys():
        try:
            p[k.lower()] = p.pop(k)
        except:
            pass
    return

def print_e(en, f):
    for i in xrange(len(en)):
        print '{:9.2f} {:9.2e}'.format(en[i], f[i])
    return

def cast_into_dic(col_names, arr):
    '''col_names is column_info, and arr is data_transitions (see below).'''
    names = col_names.split()
    return {names[i]: arr[i,:] for i in xrange(len(names))}

def calc_cooling_rate(T_s, n_E_s, n_Ion_s, N_Ion_s,
                      n_levels, n_item, n_transitions,
                      dv=1e5, stop_at_error=False):
    len_T = len(T_s)
    len_E = len(n_E_s)
    len_Ion = len(n_Ion_s)
    len_N_Ion = len(N_Ion_s)
    arr = empty((len_T, len_E, len_Ion, len_N_Ion), dtype='float')
    for i in xrange(len_T):
        T = T_s[i]
        for j in xrange(len_E):
            n_E = n_E_s[j]
            for k in xrange(len_Ion):
                n_Ion = n_Ion_s[k]
                for l in xrange(len_N_Ion):
                    N_Ion = N_Ion_s[l]
                    params = {'Tkin': T,
                      'dv_CGS': dv,
                      'dens_X_CGS': n_Ion,
                      'Ncol_X_CGS': N_Ion,
                      'H2_density_CGS': 0.0,
                      'HI_density_CGS': 0.0,
                      'oH2_density_CGS': 0.0,
                      'pH2_densty_CGS': 0.0,
                      'HII_density_CGS': 0.0,
                      'Electron_density_CGS': n_E,
                      'n_levels': n_levels,
                      'n_item': n_item,
                      'n_transitions': n_transitions}
                    lower_keys(params)
                    energies,f_occupations,data_transitions,cooling_rate = \
                        wrapper.run_one_params(**params)
                    arr[i,j,k,l] = cooling_rate/n_Ion
                    if wrapper.flag_good != 1 and stop_at_error:
                        raise ValueError('Calculation stopped due to error!')
    return arr

In [ ]:
import wrapper_my_radex
wrapper = wrapper_my_radex.myradex_wrapper
about_info = ''.join(list(wrapper.about))
column_info = ''.join(list(wrapper.column_names))

#End of header

#A test run

In [ ]:
n_levels,n_item,n_transitions = \
    wrapper.config_basic('/Users/fdu/not_synced/work/from_others/CHIANTI/toLAMDA/',
                         'Fe+_1e5K.dat', True)

In [ ]:
1e9*4e-6*1.5e13

In [ ]:
n_H = 3.5e8
X_FeII = 4.2e-6
X_E = 1e-2
T = 7e3
dv = 1e5
AU = 1.5e13
length_scale = 0.1 * AU

# Set Ncol_X_CGS to zero to have escape probability equal 1

params = {'Tkin': T,
          'dv_CGS': dv,
          'dens_X_CGS': X_FeII*n_H,
          'Ncol_X_CGS': X_FeII*n_H*length_scale,
          'H2_density_CGS': 0.0,
          'HI_density_CGS': 0.0,
          'oH2_density_CGS': 0.0,
          'pH2_densty_CGS': 0.0,
          'HII_density_CGS': 0.0,
          'Electron_density_CGS': X_E*n_H,
          'n_levels': n_levels,
          'n_item': n_item,
          'n_transitions': n_transitions}

"""The keywords to the wrapper function have to be in
lower case, so I have to lower the keys of params.
Of course you can use lower case letters from the beginning."""

lower_keys(params)

"""Do the calculation.
Return a four element tuple, whose meanings are self-evident."""

energies,f_occupations,data_transitions,cooling_rate = \
    wrapper.run_one_params(**params)

d = cast_into_dic(column_info, data_transitions)
print d.keys()
print cooling_rate

In [ ]:
params['Ncol_X_CGS'.lower()] = 0.0
energies,f_occupations,data_transitions,cooling_rate = \
    wrapper.run_one_params(**params)
print cooling_rate
params['Ncol_X_CGS'.lower()] = 1e15
energies,f_occupations,data_transitions,cooling_rate = \
    wrapper.run_one_params(**params)
print cooling_rate

In [ ]:
figure(figsize=(12,5))
plot(d['Eup'], d['tau'])
ax = gca()
ax.set_xscale('log')
ax.set_yscale('log')
set_axis_format(ax, graygrid=True)

#Batch mode

In [ ]:
N_FeII_s = array([0.0, 1e5, 1e10, 1e15])
n_FeII_s = array([1.0])
n_E_s = logspace(-2, 8, num=50)
T_s = logspace(log10(30), log10(3e4), num=40)

In [ ]:
n_levels,n_item,n_transitions = \
    wrapper.config_basic('/Users/fdu/not_synced/work/from_others/CHIANTI/toLAMDA/',
                         'Fe+_1e5K.dat', False)

In [ ]:
coo = calc_cooling_rate(T_s, n_E_s, n_FeII_s, N_FeII_s,
                        n_levels, n_item, n_transitions, dv=1e5)

In [ ]:
print coo.shape
print all(coo >= 0.0)

#Difference with different Ncol_X

In [ ]:
X = T_s
Y = n_E_s
figure(figsize=(15,10))
CTR = contourf(Y, X, log10(coo[:,:,0,0]), 200, vmin=-28.0, vmax=-10.0)
colorbar(CTR)
ax = gca()
ax.set_xscale('log')
ax.set_yscale('log')
set_axis_format(ax, graygrid=True)

In [ ]:
X = T_s
Y = n_E_s
figure(figsize=(15,10))
CTR = contourf(Y, X, log10(coo[:,:,0,3]), 200, vmin=-28.0, vmax=-10.0)
colorbar(CTR)
ax = gca()
ax.set_xscale('log')
ax.set_yscale('log')
set_axis_format(ax, graygrid=True)

#Save as a table

In [ ]:
import struct

dir_out = '/Users/fdu/'
fname_out = 'Fe+_LUT.bin'

with open(os.path.join(dir_out, fname_out), 'wb') as f:
    f.write(struct.pack('d', 2))
    f.write(struct.pack('d', len(n_E_s)))
    f.write(struct.pack('d', len(T_s)))
    log10(n_E_s).tofile(f)
    log10(T_s).tofile(f)
    log10(coo[:,:,0,0]).tofile(f)

In [ ]:
set_printoptions(linewidth=150)
print n_E_s
print T_s
print coo[0:5,0:5,0,0]

In [ ]:
ix = argmin(abs(n_E_s-1.52641797e+07))
iy = argmin(abs(T_s-1.23738791e+03))
print ix+1, n_E_s[ix]
print iy+1, T_s[iy]
print coo[iy,ix,0,0]

#Compare the curves with different Ncol_X

In [ ]:
def plt1(i1, i2):
    for i in xrange(0,len(n_E_s),2):
        plot(T_s, log10(coo[:,i,0,i1]))
    for i in xrange(0,len(n_E_s),2):
        plot(T_s, log10(coo[:,i,0,i2]), linestyle='--')
    ax = gca()
    ax.set_ylim((-30,-10))
    ax.set_xscale('log')
    ax.set_yscale('linear')
    set_axis_format(ax, graygrid=True, minorgridon=False)
    return

figure(figsize=(18,20))
subplot(1,3,1)
plt1(0, 1)
subplot(1,3,2)
plt1(0, 2)
subplot(1,3,3)
plt1(0, 3)

#Fit

In [ ]:
from scipy.optimize import curve_fit as cf

In [ ]:
def plot_many_T(X, Z):
    figure(figsize=(10,7))
    for i in xrange(Z.shape[1]):
        plot(X, Z[:,i])
    ax = gca()
    ax.set_xscale('log')
    ax.set_yscale('linear')
    set_axis_format(ax, graygrid=True)
    return

def plot_many_E(X, Z):
    figure(figsize=(10,7))
    for i in xrange(Z.shape[0]):
        plot(X, Z[i,:])
    ax = gca()
    ax.set_xscale('log')
    ax.set_yscale('linear')
    set_axis_format(ax, graygrid=True)

In [ ]:
def f1(x, a, b):
    return a*x + b

def f2(x, a, b, c, d):
    return c/(1.0+exp(-b*(x-a))) + d

def f3(x, a, b, c, d):
    xx = exp(log(10.0)*x)
    return log10(a / (1.0 + b * exp(c/xx) + d / sqrt(xx)))

In [ ]:
figure(figsize=(8,16))
popt_s = []
for i in xrange(len(T_s)):
    x = log10(n_E_s)
    y = log10(coo[i,:,0])
    plot(x, y)
    try:
        p0 = [x[len(x)/2], abs(x[len(x)/2]), 0.0, y[-1]]
        popt,_ = cf(f2, x, y, p0=p0, maxfev=1000)
        plot(x, f2(x, *popt), color='black', linestyle='--')
        popt_s.append(list(popt) + ['f2'])
    except:
        p0 = [0.0, 0.0]
        popt,_ = cf(f1, x, y, p0=p0, maxfev=1000)
        plot(x, f1(x, *popt), color='black', linestyle='--')
        popt_s.append(list(popt) + ['f1'])
    print popt

ax = gca()
ax.set_xscale('linear')
ax.set_yscale('linear')
set_axis_format(ax, graygrid=True, minorgridon=False)

#Transformation

In [ ]:
figure(figsize=(6,10))
for i in xrange(len(n_E_s)):
    plot(T_s, log10(coo[:,i,0,0]/n_E_s[i]))
ax = gca()
ax.set_xscale('log')
ax.set_yscale('linear')
set_axis_format(ax, graygrid=True, minorgridon=False)

#Fit with a two-energy system

In [ ]:
x = log10(T_s)
y = log10(coo[:,20,0,0])
#return a / (1.0 + b * exp(c/x) + d * exp(-ee*log(x)))
p0 = [1e-15, 1e1, 5e3, 1e-1]
popt,_ = cf(f3, x, y, p0=p0, maxfev=3000)
print popt

figure(figsize=(10,6))
plot(x, y)
plot(x, f3(x, *p0), linestyle='--')
ax = gca()
ax.set_ylim((-50,-5))
ax.set_xscale('linear')
ax.set_yscale('linear')
set_axis_format(ax, graygrid=True, minorgridon=False)

###Conclusion: can't fit with a two-energy system

Maybe with two two-energy systems we can fit the curve, but then there will be 10 parameters to fit.

#Calculate the cooling look-up table and save it

Batch mode

In [ ]:
import struct

def calc_and_save(dir_in=None, fname_in=None, dir_out=None, fname_out=None):
    n_levels,n_item,n_transitions = \
        wrapper.config_basic(dir_in, fname_in, False)
    N_X_s = array([0.0])
    n_X_s = array([1.0])
    n_E_s = logspace(-2, 8, num=50)
    T_s = logspace(log10(10), log10(1e6), num=60)
    
    coo_X = calc_cooling_rate(T_s, n_E_s, n_X_s, N_X_s,
                            n_levels, n_item, n_transitions, dv=1e5)

    x = log10(n_E_s)
    y = log10(T_s)
    z = log10(coo_X[:,:,0,0])
    with open(os.path.join(dir_out, fname_out), 'wb') as f:
        f.write(struct.pack('d', 2))
        f.write(struct.pack('d', len(x)))
        f.write(struct.pack('d', len(y)))
        x.tofile(f)
        y.tofile(f)
        z.tofile(f)
    print 'Some entries in the table:'
    for i in arange(0,len(x),len(y)/4+1):
        for j in arange(0,len(y),len(y)/4+1):
            print '{:12.4e}{:12.4e}{:16.8e}'.format(x[i], y[j], z[j,i])
    return

In [ ]:
calc_and_save(dir_in='/Users/fdu/not_synced/work/from_others/CHIANTI/toLAMDA/',
              fname_in='Fe+_1e5K.dat',
              dir_out='/Users/fdu/',
              fname_out='Fe+_LUT.bin')

In [ ]:
calc_and_save(dir_in='/Users/fdu/not_synced/work/from_others/CHIANTI/toLAMDA/',
              fname_in='N+_1e5K.dat',
              dir_out='/Users/fdu/',
              fname_out='N+_LUT.bin')

In [ ]:
calc_and_save(dir_in='/Users/fdu/not_synced/work/from_others/CHIANTI/toLAMDA/',
              fname_in='Si+.dat',
              dir_out='/Users/fdu/',
              fname_out='Si+_LUT.bin')

#For fun

In [ ]:
target_path = [
               '~/Downloads/',
               '~/Dropbox/',
               '~/Box Sync/new_stuff/',
               '~/MyDoc_readonly/',
               '~/not_synced/book_large/',
              ]

sizes = []
for dir_c in target_path:
    for dirname, dirnames, filenames in os.walk(os.path.expanduser(dir_c)):
        for filename in filenames:
            sizes.append(os.path.getsize(os.path.join(dirname, filename)))

sizes = array(sizes)
sizes = sizes[sizes > 0]

In [ ]:
figure(figsize=(10,6))
_ = hist(log10(sizes), bins=arange(0,9.5,0.25), color='CadetBlue', alpha=0.9)
ax = gca()
ax.set_xlabel('log$_{10}$(File size in byte)')
ax.set_ylabel('Counts per bin')
ax.set_title('All the files', fontsize=25)
yscale('log', nonposy='clip')
set_axis_format(ax, graygrid=True)

In [ ]:
target_path = [
               '~/Downloads/',
               '~/Dropbox/',
               '~/Box Sync/new_stuff/',
               '~/MyDoc_readonly/',
               '~/not_synced/book_large/',
              ]
type_to_compare = ['.pdf', '.djvu', '.djv', '.doc', '.docx', '.txt', '.epub', '.mobi', '.chm']
str_invalid_encoding = 'invalid encoding'

sizes1 = []
for dir_c in target_path:
    for dirname, dirnames, filenames in os.walk(os.path.expanduser(dir_c)):
        for filename in filenames:
            if os.path.splitext(filename)[-1].lower() in type_to_compare or str_invalid_encoding in filename:
                sizes1.append(os.path.getsize(os.path.join(dirname, filename)))

sizes1 = array(sizes1)
sizes1 = sizes1[sizes1 > 0]

In [ ]:
figure(figsize=(10,6))
_ = hist(log10(sizes), bins=arange(0,9.5,0.25), color='CadetBlue', alpha=0.9)
ax = gca()
ax.set_xlabel('log$_{10}$(File size in byte)')
ax.set_ylabel('Counts per bin')
ax.set_title('"To-be-read-by-human" files', fontsize=25)
yscale('log', nonposy='clip')
set_axis_format(ax, graygrid=True)

In [ ]:
figure(figsize=(12,6))
_ = hist(log10(sizes),  bins=arange(0,9.5,0.25), color='BurlyWood', alpha=1.0, label='All the files')
_ = hist(log10(sizes1), bins=arange(0,9.5,0.25), color='CadetBlue', alpha=1.0, label='Human readable')
ax = gca()
ax.set_xlim(0,9)
ax.set_ylim(1,1e5)
ax.set_xlabel('log$_{10}$(File size in byte)')
ax.set_ylabel('Counts per bin')
yscale('log', nonposy='clip')
set_axis_format(ax, graygrid=True)
_ = legend(loc='best', fontsize=15, framealpha=0.3)